## 環境

請填寫當前執行使用的環境。

|環境|名稱|版本|
|-|-|-|
|作業系統|Ubuntu|18.04 LTS|
|程式執行環境|colab||
|python 版本|python3|3.6.9|
|安裝環境|pip|20.0.2|

## 安裝

請填寫安裝套件需要的指令

```sh
pip install -r requirements.txt
```

## 計算資源

|計算資源|答案|
|-|-|
|使用 CPU 核心數|1|
|記憶體需求|$\leq 1$GB|
|有無使用 GPU|無|
|GPU 版本|無|
|GPU 記憶體需求|$\leq 1$GB|

## 參考資源

- 有無參考他人之實驗結果：無
- 有參考作業2的程式碼


# Colab 前置處理(一定要先執行以下所有程式)

- 取得雲端存取權限
  - 會出現網址要求登入(2次)，此時要把驗證碼複製並貼回。
- 建立相關暫存資料夾
- 將目錄移至FDA_HW3_1_F64051164
- 下載套件

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p Drive
!google-drive-ocamlfuse Drive

In [0]:
import os
#將資料目錄移至FDA_HW3_1_F64051164
os.chdir("Drive/Colab Notebooks/FDA_HW3_1_F64051164")

In [4]:
pip install -r requirements.txt

     |████████████████████████████████| 20.2MB 27.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4


In [0]:
import time
import numpy as np
import pandas as pd

EXECUTION_START_TIME = time.time() # 計算執行時間

# 讀取資料
training_set = pd.read_csv('training_set.csv')
testing_set = pd.read_csv('testing_set.csv')

# 資料前處理
## 以下是股市收盤價折線圖(2009~2017)
![結果1](http://drive.google.com/uc?export=view&id=1T32BI1KHkZK5wfoecNmfMB2DjW60MIS_)

整體而言股市呈現持續上漲的趨勢，但這並不重要，因為我們預期模組可以藉由收盤價的變化來學習，並預測未來股市的趨勢，因此我將收盤價相減取差值，然後把漲跌以0和1區分開來。

以下是資料前處理的步驟:

 - 1.將Close Price(收盤)相減後存成整數，1代表漲，0代表跌。
 - 2.用StandardScaler將資料標準化
 - 3.將資料變成DataFrame格式

In [57]:
# 資料分析與前處理
from sklearn.preprocessing import StandardScaler

#將Close Price相減後存成浮點數，1代表漲，0代表跌。
training_set['up_down'] = ((training_set['Close Price'].diff().shift(-1)) >= 0).astype('int')
testing_set['up_down'] = ((testing_set['Close Price'].diff().shift(-1)) >= 0).astype('int')

print('training_set:\n',training_set,'\n\n')
print('testing_set:\n',testing_set,'\n\n')

train_x = training_set[['Open Price', 'Close Price','High Price','Low Price','Volume']]
train_y = training_set['up_down']

test_x = testing_set[['Open Price', 'Close Price','High Price','Low Price','Volume']]
test_y = testing_set['up_down']

#用StandardScaler將資料標準化
scaler = StandardScaler()
scaler.fit(train_x)
train_x = scaler.transform(train_x)
train_x = np.transpose(train_x)


scaler.fit(test_x)
test_x = scaler.transform(test_x)
test_x = np.transpose(test_x)

#將資料變成DataFrame格式
train_x = pd.DataFrame({
    'Open Price': train_x[0],
    'Close Price': train_x[1],
    'High Price': train_x[2],
    'Low Price': train_x[3],
    'Volume': train_x[4],
})
test_x = pd.DataFrame({
    'Open Price': test_x[0],
    'Close Price': test_x[1],
    'High Price': test_x[2],
    'Low Price': test_x[3],
    'Volume': test_x[4],
})
print('training_set:\n',train_x,'\n\n')
print('testing_set:\n',test_x,'\n\n')

training_set:
              Date  Open Price  Close Price  ...  Low Price      Volume  up_down
0     02-Jan-2009      902.99       931.80  ...     899.35  4048270080        0
1     05-Jan-2009      929.17       927.45  ...     919.53  5413910016        1
2     06-Jan-2009      931.17       934.70  ...     927.28  5392620032        0
3     07-Jan-2009      927.45       906.65  ...     902.37  4704940032        1
4     08-Jan-2009      905.73       909.73  ...     896.81  4991549952        0
...           ...         ...          ...  ...        ...         ...      ...
2259  22-Dec-2017     2684.22      2683.34  ...    2678.13  1383888512        0
2260  26-Dec-2017     2679.09      2680.50  ...    2677.96  1103808384        1
2261  27-Dec-2017     2682.10      2682.62  ...    2678.91  1149108352        1
2262  28-Dec-2017     2686.10      2687.54  ...    2682.69  1126089856        0
2263  29-Dec-2017     2689.15      2673.61  ...    2673.61  1332374016        0

[2264 rows x 7 columns] 

# 所有模型

# 第一部分的Code是Logistic Regression，第二部分是DecisionTreeClassifier，最後是NN模型。

- 超參數設定與驗證演算法設定
  - LogisticRegression和DecisionTreeClassifier參數只有設定random_state，max_iter經實驗後發現不影響模型收斂。
  - 實驗後發現NN模型hidden layer設定在(5,50,20,2)的效能比較好，太少層就要花更多node達到相等精準度，太多層會有overfitting的情況。
  - random_state用來控制隨機亂數，確保每次執行結果必相同。
  - 驗證演算法採取KFold。
  - 最終測試精準度直接以答對題數所佔的百分比來比較模型之間的差異。

- 實驗結果
  - NN模型精準度最高，將近52.4%。
  - DecisionTreeClassifier精準度最低。


In [58]:
# 第一部分-Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.metrics import accuracy_score            # 匯入準確度計算工具

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

# construct the model
model = LogisticRegression(random_state=1012)         # 創造LogisticRegression模型

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
    
    model.fit(train_x_split, train_y_split)           # 訓練LogisticRegression模型
    
    train_pred_y = model.predict(train_x_split)       # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model.predict(valid_x_split)       # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))
# fit the data
answer = pd.Series(model.predict(test_x), name='answer')
result_pd = (answer == test_y)

print('testing accuracy:\n',result_pd.values.sum() / result_pd.count()*100,"%")

average train accuracy: 0.5470410771554263
    min train accuracy: 0.5422418553285477
    max train accuracy: 0.5565985643290999
average valid accuracy: 0.5463859422922893
    min valid accuracy: 0.5099337748344371
    max valid accuracy: 0.5673289183222958
testing accuracy:
 51.98412698412699 %


In [59]:
#第二部分-DecisionTreeClassifier

from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.metrics import accuracy_score            # 匯入準確度計算工具
from sklearn.tree import DecisionTreeClassifier       # import DecisionTreeClassifier

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

# construct the model
model_DecisionTree = DecisionTreeClassifier(random_state = 1012)

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
    
    model_DecisionTree.fit(train_x_split, train_y_split)       # DecisionTree
    
    train_pred_y = model_DecisionTree.predict(train_x_split)   # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model_DecisionTree.predict(valid_x_split)   # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))
# fit the data
answer = pd.Series(model_DecisionTree.predict(test_x), name='answer')
result_pd = (answer == test_y)
print('testing accuracy:\n',result_pd.values.sum() / result_pd.count()*100,"%")

average train accuracy: 1.0
    min train accuracy: 1.0
    max train accuracy: 1.0
average valid accuracy: 0.4955781515559984
    min valid accuracy: 0.46799116997792495
    max valid accuracy: 0.5386313465783664
testing accuracy:
 49.60317460317461 %


In [61]:
# NN模型
from sklearn.model_selection import KFold             # 匯入 K 次交叉驗證工具
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score            # 匯入準確度計算工具

kf = KFold(n_splits=5,                                # 設定 K 值
           random_state=1012,
           shuffle=True)
kf.get_n_splits(train_x)                              # 給予資料範圍

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

model_u = MLPClassifier(hidden_layer_sizes=(5,50,20,2), max_iter=3000, random_state=1012) # 創造NN模型(relu)

for train_index, valid_index in kf.split(train_x):    # 每個迴圈都會產生不同部份的資料
    train_x_split = train_x.iloc[train_index]         # 產生訓練資料
    train_y_split = train_y.iloc[train_index]         # 產生訓練資料標籤
    valid_x_split = train_x.iloc[valid_index]         # 產生驗證資料
    valid_y_split = train_y.iloc[valid_index]         # 產生驗證資料標籤
    
    model_u.fit(train_x_split, train_y_split)         # NN
    
    train_pred_y = model_u.predict(train_x_split)     # 確認模型是否訓練成功
    train_acc = accuracy_score(train_y_split,         # 計算訓練資料準確度
                               train_pred_y)
    valid_pred_y = model_u.predict(valid_x_split)     # 驗證模型是否訓練成功
    valid_acc = accuracy_score(valid_y_split,         # 計算驗證資料準確度
                               valid_pred_y)
    
    train_acc_list.append(train_acc)
    valid_acc_list.append(valid_acc)

print((
    'average train accuracy: {}\n' +
    '    min train accuracy: {}\n' +
    '    max train accuracy: {}\n' +
    'average valid accuracy: {}\n' +
    '    min valid accuracy: {}\n' +
    '    max valid accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list),                           # 輸出最高訓練準確度
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))
# fit the data
answer = pd.Series(model_u.predict(test_x), name='answer')
result_pd = (answer == test_y)
print('testing accuracy:\n',result_pd.values.sum() / result_pd.count()*100,"%")


average train accuracy: 0.5472619496015885
    min train accuracy: 0.5422418553285477
    max train accuracy: 0.5565985643290999
average valid accuracy: 0.5472689444997949
    min valid accuracy: 0.5099337748344371
    max valid accuracy: 0.5673289183222958
testing accuracy:
 52.38095238095239 %


In [14]:
EXECUTION_END_TIME = time.time() # 計算執行時間
print('total execution time: {}'.format(EXECUTION_END_TIME - EXECUTION_START_TIME))

total execution time: 379.66858863830566


# 改用2019年的資料測試
- 請執行以下程式碼之後，再從資料前處理的區塊開始執行就好。

In [0]:
# 讀取資料
training_set = pd.read_csv('training_set.csv')
testing_set = pd.read_csv('testing_set_2019.csv')

# 執行結果與討論

#### 改用2019年的資料測試的結果
- DecisionTreeClassifier的結果仍然是3個模型之間最低的(50.8%)。
- LogisticRegression與NN模型精準度相同(59.12%)。

#### 結論
- 就這次的結果來看，在資料量沒有非常多的情況下(僅2000多筆)，LogisticRegression與層數不多的NN模型的強度非常接近(可能是因為LogisticRegression算是ANN的前身)，但是NN模型在兩次的測試之中皆保持第一名的成績。
- DecisionTreeClassifier有overfitting的現象，可能是因為沒辦法學到更深層股市漲跌的feature，但是也不代表DecisionTreeClassifier比較差，只是這些股市的資料並不適合DecisionTreeClassifier學習。
- 我有嘗試使用1~2層的hidden layer，但是因為要使用更多的node才能達到相等的精確度，因此我採取多一點層數的策略以減少運算資源，但是層數太多又會導致overfitting，所以我將hidden layer層數設為4。
- 嚴格來說NN模型在這次的資料表現最好，其次是LogisticRegression，最後是DecisionTreeClassifier。
